# Community Detection

The scope of this notebook is to provide some examples of how SurpriseMeMore can
be used to detect communities in binary and weighted network. We use the undirected
*les miserables* character network in the next examples, but all the applications
 that we are going to see can be generalized to the case of directed networks.

### Important

All the methods we present are heuristic, we suggest then to run it more than once
to increase the chance to find the best partitioning.

In [8]:
from surprisememore import Undirected_Graph_Class as ug
import numpy as np
import networkx as nx
import pandas as pd

aux_path = 'out.moreno_lesmis_lesmis'
prova = np.loadtxt(aux_path,comments='%')
edgelist = pd.DataFrame(prova,columns=['source','target','weight'])
network = nx.from_pandas_edgelist(edgelist,source='source',target='target',edge_attr=True,create_using=nx.Graph)
adjacency_matrix = nx.to_numpy_array(network)

We initialize our SupriseMeMore **UndirectedGraph** object with the adjacency
matrix. The available options are adjacency matrix or edgelist. We suggest to
use numpy array in both cases.

## Basic Usage

In [9]:
graph = ug.UndirectedGraph(adjacency_matrix)

Now that our graph istance is initalized we can run discrete community
detection by tapping the following command.

In [12]:
graph.run_discrete_community_detection()

# The optimal partitioning is given by
print(graph.solution)
# This is a numpy array where the membership of each node is stored. Node with
# the same membership are in the same cluster.

# The relative logsurprise is
print(graph.log_surprise)
# and the associated p-value (surprise) is
print(graph.surprise)

100%|██████████| 254/254 [00:00<00:00, 30605.68it/s]

[17  4  4  4 14 15 16  4  4 11  0 19 20 18 21 22  2  1  1  0  0  5  3  8
 12 10  3  3  3  3  3 13  5  0  0  0  0  0  0  1  1  1  1 28  2  2  2  2
  2  2  2  8  1  1  1  9 27  1 26  5  9  0  0  6  6  0  0  0  0  0  0  7
  7 24 25 23 29]
349.86182922756353
0.0


### Important

Everytime that you run the algorithm the values *solution*, *log_surprise* and
*surprise* are overwritten.

## Community Detection Arguments

SurpriseMeMore community detection methods allow the user to pass arguments
specifying the used method to use and other useful options. In what follows we
will brifly discuss these options.

### method

First we can specify the clustering method: **aglomerative** or **fixed-clusters**.

**Aglomerative** method starts with all the nodes in different partitions and then start
to merge these until no further improvements are possible.

**Fixed-clusters** solves the partioning problem with a fixed number of clusters
(defined by the user). If *fixed-clusters* is passed, the number of cluster argument
 (*num_clusters*) must be specified.

The default value is *aglomerative*.

### initial_guess

The user can pass its own initial guess to the algorithm (pay attention that
has to be a proper initial guess) or use one of the implemented one.

* *random*: membership is assigned to nodes randomly. If the *method* is aglomerative
it doesn't affect the initial guess;

* *common-neigh-weak*: nodes are in the same initial partition based on the fraction
of common-neighbours. The condtion of aggregation is more relaxed.

* *common-neigh-strong*: nodes are in the same initial partition based on the fraction
of common-neighbours. The condtion of aggregation is more strictly.

The default value is *random*.

### weighted

This argument has to be used when we initialize a UndirectedGraph (DirectedGraph)
instance that is weighted. In that case, if we want to run binary community detection
we must specify *weighted*=False.

```
    graph.run_discrete_community_detection('weighted'=False)
```

The above snippet of code run binary community detection on les miserables graph.

The *weighted* argument is just for discrete community detection methods,
in the case of enhanced or continuous community detection there is no binary
version of the algorithm.

The default value is *None*, the algorithm will choose the proper method for
the network:

* weighted network --> weighted surprise;

* binary netowrk --> binary surprise;

### num_sim

Number of times the algorithm will run over all the links trying to improve the
partioning. If no improvements are detected for 10 times in a row then the algorithm
stops.

The default value is 2.

##  Discrete Community Detection

### Binary

In [14]:
# An example of how we can run it
graph.run_discrete_community_detection(weighted=False,
                                       initial_guess="random",
                                       num_sim=3)
print("Aglomerative solution", graph.solution)

# for the fixed clusters one
graph.run_discrete_community_detection(method="fixed-clusters",
                                       num_clusters=4,
                                       weighted=False,
                                       initial_guess="random",
                                       num_sim=3)
print("Fixed clusters solution", graph.solution)

100%|██████████| 254/254 [00:00<00:00, 8074.04it/s]

Aglomerative solution [16  5  5  5 13 18 19 20 21  5  3 17 23  8  3  8  2  1  1  4  1  6  3 12
 15 14  3  3  3  3  3  7  6  0  4  4  0  0  0  1  1  1  1  7  2  2  2  2
  2  2  2 12  1  1 22 10 25  1  4  6 10  0  0  9  9  0  0  0  0  0  0 11
 11  0 24  4 26]
Fixed clusters solution [0 1 1 2 2 0 0 0 3 2 0 3 3 2 3 1 1 1 0 0 0 1 0 1 0 1 0 0 1 3 2 2 3 0 0 1 0
 1 3 1 0 3 1 2 1 1 2 0 0 0 1 1 0 0 2 0 2 2 0 3 2 1 2 3 1 1 0 1 1 0 0 3 2 1
 2 0 2]


### Weighted

In [7]:
# An example of how we can run it
graph.run_discrete_community_detection(weighted=True,
                                       initial_guess="random",
                                       num_sim=3)
print("Aglomerative solution", graph.solution)
# for the fixed clusters one
graph.run_discrete_community_detection(method="fixed-clusters",
                                       num_clusters=3,
                                       weighted=True,
                                       initial_guess="random",
                                       num_sim=3)
print("Fixed clusters solution", graph.solution)

## Enhanced Community Detection

For more details about the differences between enhanced methods and discrete one
 read the relative paper (you can find the link in the readme).

In [7]:
# An example of how we can run it
graph.run_enhanced_community_detection(initial_guess="common-neigh-weak",
                                       num_sim=4)
print("Aglomerative solution", graph.solution)
# for the fixed clusters one
graph.run_enhanced_community_detection(method="fixed-clusters",
                                       num_clusters=4,
                                       initial_guess="common-neigh-weak",
                                       num_sim=4)
print("Fixed clusters solution", graph.solution)

## Continuous Community Detection

Continuos community detection has to be used when your graph has continuos weights.
It requires integration then it is way slower.

In [7]:
# An example of how we can run it
graph.run_continuous_community_detection(initial_guess="common-neigh-weak",
                                         num_sim=3)
print("Aglomerative solution", graph.solution)
# for the fixed clusters one
graph.run_enhanced_community_detection(method="fixed-clusters",
                                       num_clusters=5,
                                       initial_guess="common-neigh-weak",
                                       num_sim=4)
print("Fixed clusters solution", graph.solution)